# Hypothesentests 1

### Daten aus der CSV-Datei lesen

Nun benutzen wir pandas um die [CSV-Datei](https://de.wikipedia.org/wiki/CSV_(Dateiformat)) einzulesen. Damit das funktioniert muss vorher noch das pandas-Paket mit dem `import`-Befehl unter dem Namen `pd` verfügbar gemacht werden.

Dabei wird ein sogenannter [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) mit dem Namen `nba` erzeugt. Vereinfacht kann man sich darunter eine Tabelle mit Spalten und Zeilen vorstellen. DataFrames sind **die** zentralen Datentypen in pandas.   

Mit dem Befehl `type()` kann der Datentyp eines jeden Python-Objekts ausgegeben werden. 

In [212]:
import pandas as pd
import numpy as np
nba = pd.read_csv("nba_all_elo.csv")
type(nba)
# Expected:
# <class 'pandas.core.frame.DataFrame'>

pandas.core.frame.DataFrame

In [213]:
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN


# Aufgabe: Finals
![finals](./finals.png)

Seit 2005 werden die Playoffs, das Finalturnier der NBA, in einem Wettbewerb im Best-of-Seven Moduls gespielt.
Eine Finalserie hat also zwischen 4 und 7 Spielen zwischen einer Spielpaarung. Dabei werden die ersten vier Spiele je 2 Spiele am Heimort der ersten Mannschaft und anschließend zwei Spiele am Heimort der anderen Mannschaft gespielt. Danach wird der Spielort pro Spiel gewechselt. Folglich startet eine Mannschaft mit zwei Auswärtsspielen in eines solche Finalserie. Siehe: https://en.wikipedia.org/wiki/NBA_playoffs

![finals](./finals_PHX_DAL.png)



Da im Sport Erfahrungen, Binsenweisheiten und andere Mutmaßungen sehr weit verbreitet ist können man sich folgenden Ausspruch vorstellen:

>"Das Auswärtsteam wird eine Serie eher gewinnen wenn Sie das erste Auswärtsspiel (Spiel 1 der Serie) gewinnt und das zweite verliert als anders herum."

Da wir die Daten zu allen Spielen haben lässt sich das ja nun eigentlich sehr einfach nachprüfen.

In den nächsten beiden Einheiten werden wird die Daten soweit aufbereiten um diese Hypothese zu testen. Dazu müssen wir nun erst einmal die Serien in den Daten nachbilden.


### 1. letzte Vorbereitungsschritte...

#### Ergebnisse für alle ersten und zweiten Spiele einer Serie finden und in eine neue Spalte `game1_2` eintragen. 

Das Format ist `W_L`, `W_W`, `L_W` bzw. `L_L`. Die Ergebnisse sollten ab dieser Stelle sinnvollerweise auf Kopien gefiltert werden. 
Außerdem wird nun nur noch eine Zeile pro Serie benötigt. 

Der DataFrame soll zum Schluß die folgende Spalten haben: 
`year_id`, `date_game` (= erstes Spiel), `team_id`, `fran_id`, `opp_id`, `opp_fran`, `game_location`, `num_series_wins`, `opp_series_wins`, `series_winner` und `game1_2`.

Um die Ergebnisse der Spiele 1 und 2 zusammen zu bringen müssen in diesem Fall Werte aus zwei verschiedene Zeilen gelesen werden. Dazu könnten Sie zum Beispiel die `transform()` Funktion in verbindung mit einer lambda-Funktion verbinden, die die Zeichenketten verbindet. Siehe: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html?highlight=transform#pandas.DataFrame.transform 

Eine weitere Möglichkeit wäre die Verwendung von `merge()`, um denn Data Frame mit per cross-join "sich selbst" zu verbinden, und anschließend alle überflüssigen Zeilen herauszufiltern. 

Schritt 1:

Series | year_id | num_series_x | ... | game_result_x | num_series_y | game_result_y 
---|---|---|---|---|---|---
HOU_DAL | 2005 | 1.0 | ... | W | 1.0 | W 
HOU_DAL | 2005 | 2.0 | ... | L | 1.0 | W 
HOU_DAL | 2005 | 1.0 | ... | W | 2.0 | L 
HOU_DAL | 2005 | 2.0 | ... | L | 2.0 | L 

Schritt 2:

Series | year_id | num_series_x | ... | game_result_x | num_series_y | game_result_y 
---|---|---|---|---|---|---
HOU_DAL | 2005 | 1.0 | ... | W | 2.0 | L 

Schritt 3:

Series | year_id |  ... | game1_2
---|---|---|---
HOU_DAL | 2005 |  ... | W_L 



### 2. Die erste Auswertung

Ermitteln Sie für die zu untersuchenden Fälle "W_L" und "L_W"  aus Sicht des Auswärtsteams folgende Kennzahlen: 
- Anzahl der Serien
- gewonnene Serien
- verlorene Serien
- Anteil der Siege in %


Das Ergebnis hat zwei Zeilen in denen die vier Kennzahlen und der jeweilige Fall dargestellt werden. 

Was können Sie zum Ergebnis sagen? Hat sich die Vermutung zum Einfluß der Siegreihenfolge bestätigt?


### 3. Eine erweiterte Untersuchung

Erweitern Sie den Bereich der untersuchten Spiele auf alle Best-of-7-Serien Seit 1984 (inkl.). Achtung: Hier wurden teilweise auch Best-Of-3 und Best-Of-5 Serien gespielt. Diese sind hier auszuschließen.

Ermitteln Sie die Kennzahlen zu dem erweiterten Datenbestand analog zur vorigen Aufgabe.